## Read rules from all cubes and sort cubes by some metrics (Number rows, Number feeders,... )

In [22]:
import pandas as pd
import configparser
import TM1py
from TM1py.Services import TM1Service
config = configparser.ConfigParser()
config.read(r'../config.ini')
tm1 = TM1Service(**config['tm1srv02'])

In [10]:
from TM1py.Services import TM1Service
from TM1py.Utils.Utils import build_pandas_dataframe_from_cellset
from TM1py.Utils.Utils import build_cellset_from_pandas_dataframe

In [2]:
dimension_name = 'Region'
tm1.dimensions.get(dimension_name=dimension_name).name
tm1_master = TM1Service(**config['tm1srv01'])
tm1_other = TM1Service(**config['tm1srv02'])

# while True:
dimension_master = tm1_master.dimensions.get(dimension_name="dimension_test")

In [29]:
cube_name=f'ElementAttributes_{dimension_master.name}'
cube_name

'ElementAttributes_dimension_test'

In [75]:
cube_name = '}ElementAttributes_dimension_test'
cube_dimensions_names = tm1.cubes.get_dimension_names(cube_name=cube_name)
tm1.cubes.views.create()

In [58]:
elements_ = []
for dimension_name in cube_dimensions_names:
    elements = []
    hierarchies = tm1.hierarchies.get_all_names(dimension_name=dimension_name)
    elements += tm1.elements.get_element_names(dimension_name=dimension_name,hierarchy_name=hierarchies[0])
    elements_ += [elements]

In [29]:
tm1.cubes.get_all_names()

['Commissions',
 'Customer_Churn',
 'Expenses',
 'Franchise_Revenue',
 'Franchise_Revenue2',
 'Franchise_Revenue3',
 'Income_Statement',
 'Price_and_Cost',
 'Rules',
 'Sales_Plan',
 'Spread_Profile_SubsByMonth',
 'Spread_Profile_Subsidiary',
 'Spread_Profiles',
 'zRate',
 'zTime_Navigation',
 '}appClientProperties',
 '}Capabilities',
 '}CellSecurity_}tp_application_root_permissions',
 '}ChoreSecurity',
 '}ClientCAMAssociatedGroups',
 '}ClientGroups',
 '}ClientProperties',
 '}ClientSettings',
 '}CubeAttributes',
 '}CubeCaptions',
 '}CubeDrill_Commissions',
 '}CubeDrill_Sales_Plan',
 '}CubeProperties',
 '}CubeSecurity',
 '}CubeSecurityProperties',
 '}DimensionAttributes',
 '}DimensionCaptions',
 '}DimensionProperties',
 '}DimensionSecurity',
 '}ElementAttributes_Channels',
 '}ElementAttributes_Churn_Measures',
 '}ElementAttributes_Commission_Measures',
 '}ElementAttributes_Currency',
 '}ElementAttributes_CurrencyConvertTo',
 '}ElementAttributes_CurrencyType',
 '}ElementAttributes_dimensi

In [73]:
dataframe = tm1.cubes.cells.execute_view_raw(cube_name='Sales_Plan',view_name="TM1View",skip_zeros=False)

In [81]:
data = tm1.cubes.cells.execute_view(cube_name='Sales_Plan', view_name='TM1View',skip_zeros=False)
df = build_pandas_dataframe_from_cellset(data, multiindex=False, sort_values=False)
data_list = [{item: value} for item,value in data.items()]
df = df.rename({'Sales_Plan_Measures':'KPI'},axis=1)
df_md = df.to_markdown(index=False)
print(df_md)

| Subsidiaries       | Channels     | Products     | Months     | Versions         | KPI      |   Values |
|:-------------------|:-------------|:-------------|:-----------|:-----------------|:---------|---------:|
| GO AMERICAS REGION | ALL CHANNELS | Cooking Gear | Jan        | Budget Version 1 | Quantity |    20218 |
| GO AMERICAS REGION | ALL CHANNELS | Cooking Gear | Feb        | Budget Version 1 | Quantity |    27634 |
| GO AMERICAS REGION | ALL CHANNELS | Cooking Gear | Mar        | Budget Version 1 | Quantity |    25927 |
| GO AMERICAS REGION | ALL CHANNELS | Cooking Gear | Apr        | Budget Version 1 | Quantity |    12340 |
| GO AMERICAS REGION | ALL CHANNELS | Cooking Gear | May        | Budget Version 1 | Quantity |     7615 |
| GO AMERICAS REGION | ALL CHANNELS | Cooking Gear | Jun        | Budget Version 1 | Quantity |    16540 |
| GO AMERICAS REGION | ALL CHANNELS | Cooking Gear | Jul        | Budget Version 1 | Quantity |     5841 |
| GO AMERICAS REGION | ALL CHANNELS |

In [88]:
dataframe.to_csv('..\Outputs\exports_csv',index=False)

<>:1: SyntaxWarning: invalid escape sequence '\O'
<>:1: SyntaxWarning: invalid escape sequence '\O'
C:\Users\ErwanTang\AppData\Local\Temp\ipykernel_2348\841774845.py:1: SyntaxWarning: invalid escape sequence '\O'
  dataframe.to_csv('..\Outputs\exports_csv',index=False)


In [22]:
dimension_name = 'Region'
for hierarchy in tm1.dimensions.get(dimension_name=dimension_name):
    print('Hierarchy Name: {}'.format(hierarchy.name))
    start = 0
    current_level = start
    current_type = 'Consolidated'
    previous_type = current_type[:]
    # iterate through Elements in hierarchy
    for element in hierarchy:
        current_type = element.element_type
        if str(previous_type) == 'Consolidated':
            current_level += 1
        else:
            if str(current_type) == 'Consolidated':
                current_level -= 1
        level_string = '--'*current_level
        previous_type = current_type
        print(level_string,element.name)

Hierarchy Name: Region
-- World
---- Europe
------ Scandinavia
-------- Denmark
-------- Norway
-------- Sweden
------ Benelux
-------- Belgium
-------- Luxemburg
-------- Netherlands
------ Islands
-------- Great Britain
-------- Ireland
------ Central Europe
-------- France
-------- Germany
------ Iberia
-------- Portugal
-------- Spain
------ Southern Europe
-------- Greece
-------- Italy
------ Americas
-------- North America
---------- Canada
---------- Mexico
---------- United States
-------- South America
---------- Argentina
---------- Brazil
---------- Chile
---------- Uruguay
Hierarchy Name: By Vice President
-- Uruguay
-- Chile
-- Brazil
-- Argentina
-- United States
-- Mexico
-- Canada
-- Italy
-- Greece
-- Spain
-- Portugal
-- Germany
-- France
-- Ireland
-- Great Britain
-- Netherlands
-- Luxemburg
-- Belgium
-- Sweden
 VP2
-- Norway
 VP1
-- Denmark
 All By Vice President
Hierarchy Name: H2 Reorg
-- Uruguay
-- Chile
-- Brazil
 All H2 Reorg
-- Americas
---- South America
-

In [60]:
reordered_parents

{'South America': ['Uruguay', 'Chile', 'Brazil'],
 'All H2 Reorg': ['Americas', 'Europe'],
 'Americas': ['South America', 'North America'],
 'North America': ['Argentina', 'United States', 'Mexico', 'Canada'],
 'Southern Europe': ['Italy', 'Greece'],
 'Europe': ['Southern Europe',
  'Iberia',
  'Central Europe',
  'Islands',
  'Benelux',
  'Scandinavia'],
 'Iberia': ['Spain', 'Portugal'],
 'Central Europe': ['Germany', 'France'],
 'Islands': ['Ireland', 'Great Britain'],
 'Benelux': ['Netherlands', 'Luxemburg', 'Belgium'],
 'Scandinavia': ['Sweden', 'Norway', 'Denmark']}

In [59]:
consolidated_elements = {}
for key,val in reordered_parents.items():
    for val2 in reordered_parents[key]:
        if val2 in reordered_parents.keys() :
            consolidated_elements += reordered_parents[val2]

        #     val2.replace(key,val)
consolidated_elements

['South America',
 'North America',
 'Southern Europe',
 'Iberia',
 'Central Europe',
 'Islands',
 'Benelux',
 'Scandinavia',
 'Uruguay',
 'Chile',
 'Brazil',
 'Argentina',
 'United States',
 'Mexico',
 'Canada',
 'Italy',
 'Greece',
 'Spain',
 'Portugal',
 'Germany',
 'France',
 'Ireland',
 'Great Britain',
 'Netherlands',
 'Luxemburg',
 'Belgium',
 'Sweden',
 'Norway',
 'Denmark']

In [58]:
dict_parents = tm1.elements.get_parents_of_all_elements(dimension_name=dimension_name,hierarchy_name='H2 Reorg')
reordered_parents = {}
for key,val in dict_parents.items():
    if val:
        if val[0] not in reordered_parents.keys():
            reordered_parents[val[0]] = [key]
        else:
            reordered_parents[val[0]] += [key]
    else:
        root = key

print(root)

All H2 Reorg


In [100]:
dimension_name = 'Region'

for hierarchy in tm1.dimensions.get(dimension_name=dimension_name):
    # print(hierarchy.name)
    parents_dict = dict(hierarchy.edges).keys()
    new_dict = {}
    keys = [tup[0] for tup in parents_dict]
    values = set([tup[1] for tup in parents_dict])
    # print(keys)
    # print(values)
    # print(list(parents_dict))
    # print(tm1.elements.get_element_attributes(dimension_name=dimension_name, hierarchy_name=hierarchy.name))
    for attribute in tm1.elements.get_element_attributes(dimension_name=dimension_name, hierarchy_name=hierarchy.name):
        print(attribute)
    # for attribute in tm1.elements.get_element_attributes(dimension_name=dimension_name, hierarchy_name=hierarchy.name):
    #     print(tm1.elements.get_attribute_of_elements(dimension_name=dimension_name, hierarchy_name=hierarchy.name, elements=tm1.elements.get_element_names(dimension_name=dimension_name, hierarchy_name=hierarchy.name),attribute=attribute))

{"Name": "H2 Reorg", "Type": "String"}
{"Name": "By Vice President", "Type": "String"}
{"Name": "H2 Reorg", "Type": "String"}
{"Name": "By Vice President", "Type": "String"}
{"Name": "H2 Reorg", "Type": "String"}
{"Name": "By Vice President", "Type": "String"}
{"Name": "H2 Reorg", "Type": "String"}
{"Name": "By Vice President", "Type": "String"}
{"Name": "H2 Reorg", "Type": "String"}
{"Name": "By Vice President", "Type": "String"}


In [64]:
for hierarchy in tm1.dimensions.get(dimension_name='Region'):
    print("Hierarchy")
    print(hierarchy.name)
    consolidated_names = tm1.elements.get_edges(dimension_name='Region',hierarchy_name=hierarchy.name)
    for consolidation in consolidated_names:
        print(f'|- {consolidation[1]}')
        consolidation_elements = tm1.elements.get_parents_of_all_elements(dimension_name='Region',hierarchy_name= hierarchy.name,consolidation=consolidation[1])
        print(consolidation_elements)
        element_parents = tm1.elements.get_parents(dimension_name='Region',hierarchy_name= hierarchy.name,consolidation=consolidation[1])
        
        element_index = 0
        while element_parents[element_parents.keys()[element_index]] == current_parent:
            print(element_parents.keys()[element_index])
        print(tm1.elements.get_levels_count(dimension_name='Region',hierarchy_name= hierarchy.name))
        
        # for element in consolidation_elements:
        #     print(f'  |- {element[1]}')

Hierarchy
Region
|- Europe
{'World': [], 'Europe': ['World'], 'Scandinavia': ['Europe'], 'Denmark': ['Scandinavia'], 'Norway': ['Scandinavia'], 'Sweden': ['Scandinavia'], 'Benelux': ['Europe'], 'Belgium': ['Benelux'], 'Luxemburg': ['Benelux'], 'Netherlands': ['Benelux'], 'Islands': ['Europe'], 'Great Britain': ['Islands'], 'Ireland': ['Islands'], 'Central Europe': ['Europe'], 'France': ['Central Europe'], 'Germany': ['Central Europe'], 'Iberia': ['Europe'], 'Portugal': ['Iberia'], 'Spain': ['Iberia'], 'Southern Europe': ['Europe'], 'Greece': ['Southern Europe'], 'Italy': ['Southern Europe'], 'Americas': ['World'], 'North America': ['Americas'], 'Canada': ['North America'], 'Mexico': ['North America'], 'United States': ['North America'], 'South America': ['Americas'], 'Argentina': ['South America'], 'Brazil': ['South America'], 'Chile': ['South America'], 'Uruguay': ['South America']}
4
|- Americas
{'World': [], 'Europe': ['World'], 'Scandinavia': ['Europe'], 'Denmark': ['Scandinavia'],

In [35]:
for hierarchy in dimension:
            print('Hierarchy Name: {}'.format(hierarchy.name))
            # iterate through Elements in hierarchy
            for element in hierarchy:
                print(['Element Name: {} Index: {} Type: {}'.format(element.name, str(element.index), element.element_type)])


AttributeError: 'str' object has no attribute 'name'

In [66]:
mdx = "SELECT " \
          "NON EMPTY {TM1SUBSETALL( [}Clients] )} on ROWS, " \
          "NON EMPTY {TM1SUBSETALL( [}Groups] )} ON COLUMNS " \
          "FROM [}ClientGroups]"
cube_content = tm1.cubes.cells.execute_mdx(mdx, ['Value'])

In [29]:
all_cubes = tm1.cubes.get_all()
dimension = 'Region'
for cube in all_cubes:
    if dimension in cube.dimensions:
          print(cube.name)

Sales
Sales_Test
TEST
}ElementAttributes_Region
}ElementSecurity_Region
}PickList_TEST


In [69]:
tm1.cubes.cells.execute_mdx_dataframe(mdx=mdx)['}Clients']

0         adel.hamache
1                Admin
2         amavi.atigla
3       aurele.nassara
4         badr.bouhlal
5      bruno.taboument
6           elhadj.sow
7           erwan.tang
8       franck.luciani
9             Hugo.lim
10        jamil.fadile
11      Jennifer.ardon
12        karima.barki
13    laurent.mariotte
14        ludovic.sant
15      mariama.diallo
16     marianne.plesse
17     mathieu.botquin
18      mehdi.taheraly
19           test.user
20      Virginie.jumel
21      Yasmine.ghanem
22       zied.benhmida
Name: }Clients, dtype: object

In [51]:
client_groups_content = tm1.cubes.cells.execute_view_dataframe(cube_name='}ClientGroups',view_name='All')

TM1pyRestException: Text: '{"error":{"code":"278","message":"'All' can not be found in collection of type 'View'."}}' - Status Code: 404 - Reason: 'Not Found' - Headers: {'Content-Length': '103', 'Connection': 'keep-alive', 'Content-Encoding': 'gzip', 'Cache-Control': 'no-cache', 'Content-Type': 'application/json; charset=utf-8', 'OData-Version': '4.0'}

In [21]:
client_groups_content

NameError: name 'client_groups_content' is not defined

In [57]:
used_groups

{'ADMIN',
 'CFO',
 'Contribute_Americas',
 'Contribute_Central_Europe',
 'DataAdmin',
 'Education',
 'Executive',
 'Manager',
 'Review_Americas',
 'Review_Europe',
 'View_Company',
 '}tp_Everyone'}

In [2]:
from TM1py.Utils import get_all_servers_from_adminhost

tm1_instances_on_server = get_all_servers_from_adminhost('91.236.254.119',None,True)

In [4]:
for tm1_instance in tm1_instances_on_server:
    print(tm1_instance.name)

CarSales_Test
Datatilt - Interne
Datatilt - Planning Equipe
ESSP_Cost_Sheet_dev
IBM - Demo2023
Sample - SData
SampleOutdoors ETA
Socfin
TK
TM1 Login Server


In [7]:
my_new_process = TM1py.Objects.Process(name='newprocess')

In [34]:
my_new_process.data_procedure

'#****Begin: Generated Statements***\r\n#****End: Generated Statements****\r\nB'

In [37]:
def edit_process_code(process,section):
    new_code = input('new code:')
    if section.lower() == 'prolog':
        my_new_process.prolog_procedure = my_new_process.prolog_procedure[:73]+new_code
    elif section.lower() == 'metadata':
        my_new_process.metadata_procedure = my_new_process.metadata_procedure[:73]+new_code
    elif section.lower() == 'data':
        my_new_process.data_procedure = my_new_process.data_procedure[:73]+new_code
    elif section.lower() == 'epilog':
        my_new_process.epilog_procedure = my_new_process.epilog_procedure[:73]+new_code

In [39]:
edit_process_code(my_new_process,'metadata')
my_new_process

Process:{"Name": "newprocess", "PrologProcedure": "#****Begin: Generated Statements***\r\n#****End: Generated Statements****\r\n", "MetadataProcedure": "#****Begin: Generated Statements***\r\n#****End: Generated Statements****\r\nazerty", "DataProcedure": "#****Begin: Generated Statements***\r\n#****End: Generated Statements****\r\nB", "EpilogProcedure": "#****Begin: Generated Statements***\r\n#****End: Generated Statements****\r\n", "HasSecurityAccess": false, "UIData": "CubeAction=1511\fDataAction=1503\fCubeLogChanges=0\f", "DataSource": {"Type": "None"}, "Parameters": [], "Variables": [], "VariablesUIData": []}

In [19]:
len('#****Begin: Generated Statements***\r\n#****End: Generated Statements****\r\n')

73

In [42]:
tm1.elements.get_levels_count(dimension_name='Region',hierarchy_name='Region')

4

In [43]:
tm1.elements.get_elements_by_level(dimension_name='Region',hierarchy_name='Region', level= 1)


['Scandinavia',
 'Benelux',
 'Islands',
 'Central Europe',
 'Iberia',
 'Southern Europe',
 'North America',
 'South America']

In [48]:
dimension_name = 'Region'

In [44]:
tm1.elements.get_level_names(dimension_name='Region',hierarchy_name='Region')

['level003', 'level002', 'level001', 'level000']

In [46]:
tm1.elements.get_members_under_consolidation(dimension_name='Region',hierarchy_name='Region',consolidation='World',max_depth=1)

['Europe', 'Americas']

In [54]:
hierarchy = tm1.hierarchies.get(dimension_name=dimension_name, hierarchy_name=dimension_name)

In [72]:
levels_count = tm1.elements.get_levels_count(dimension_name=dimension_name,hierarchy_name=hierarchy.name)
print(levels_count)
elements_level = {}
print('Hierarchy Name: {}'.format(hierarchy.name))
for level in range(levels_count):
    elements_by_level = tm1.elements.get_elements_by_level(dimension_name=dimension_name,hierarchy_name=hierarchy.name, level=level)
    elements_level[level] = elements_by_level
for max_conso in tm1.elements.get_elements_by_level(dimension_name=dimension_name,hierarchy_name=hierarchy.name, level=levels_count-1):
    print([
        level1 for level1 in [level0 for level0 in tm1.elements.get_members_under_consolidation(dimension_name='Region',hierarchy_name='Region',consolidation=max_conso,max_depth=1)
        ]])

4
Hierarchy Name: Region
['Europe', 'Americas']


In [70]:
for max_conso in tm1.elements.get_elements_by_level(dimension_name=dimension_name,hierarchy_name=hierarchy.name, level=levels_count-1):
    print(max_conso)
    print([level0 for level0 in tm1.elements.get_members_under_consolidation(dimension_name='Region',hierarchy_name='Region',consolidation='World',max_depth=1)])

World
['Europe', 'Americas']


In [77]:
all_levels = []
for max_conso in tm1.elements.get_elements_by_level(dimension_name=dimension_name,hierarchy_name=hierarchy.name, level=levels_count-1):
        all_levels += [{max_conso:tm1.elements.get_members_under_consolidation(dimension_name='Region',hierarchy_name='Region',consolidation=max_conso,max_depth=1)}]
        for level0 in tm1.elements.get_members_under_consolidation(dimension_name='Region',hierarchy_name='Region',consolidation=max_conso,max_depth=1):
            all_levels += [{level0:tm1.elements.get_members_under_consolidation(dimension_name='Region',hierarchy_name='Region',consolidation=level0,max_depth=1)}]
            for level1 in tm1.elements.get_members_under_consolidation(dimension_name='Region',hierarchy_name='Region',consolidation=level0,max_depth=1):
                all_levels += [{level1:tm1.elements.get_members_under_consolidation(dimension_name='Region',hierarchy_name='Region',consolidation=level1,max_depth=1)}]
all_levels

[{'World': ['Europe', 'Americas']},
 {'Europe': ['Scandinavia',
   'Benelux',
   'Islands',
   'Central Europe',
   'Iberia',
   'Southern Europe']},
 {'Scandinavia': ['Denmark', 'Norway', 'Sweden']},
 {'Benelux': ['Luxemburg', 'Netherlands', 'Belgium']},
 {'Islands': ['Great Britain', 'Ireland']},
 {'Central Europe': ['France', 'Germany']},
 {'Iberia': ['Portugal', 'Spain']},
 {'Southern Europe': ['Greece', 'Italy']},
 {'Americas': ['North America', 'South America']},
 {'North America': ['Canada', 'Mexico', 'United States']},
 {'South America': ['Argentina', 'Brazil', 'Chile', 'Uruguay']}]

In [81]:
def replace_par_with_child(children_list):
    for i in range(len(children_list)):
        children_list[i] = {children_list[i]:tm1.elements.get_members_under_consolidation(dimension_name='Region',hierarchy_name='Region',consolidation=children_list[i],max_depth=1)}
    

In [82]:
L = ['Europe', 'Americas']
replace_par_with_child(L)
L


[{'Europe': ['Scandinavia',
   'Benelux',
   'Islands',
   'Central Europe',
   'Iberia',
   'Southern Europe']},
 {'Americas': ['North America', 'South America']}]

In [89]:
par_dic = tm1.elements.get_parents_of_all_elements(dimension_name=dimension_name,hierarchy_name=dimension_name)
par_dic
dataframe = tm1.elements.get_elements_dataframe(dimension_name=dimension_name,hierarchy_name=dimension_name)

c:\Users\ErwanTang\AppData\Local\Programs\Python\Python312\Lib\site-packages\TM1py\Services\ElementService.py:381: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_data[cols_to_format] = df_data[cols_to_format].applymap(lambda x: '{:.6f}'.format(x))


In [102]:
level_names = tm1.elements.get_level_names(dimension_name=dimension_name,hierarchy_name=dimension_name)
attribute_names = tm1.elements.get_element_attribute_names(dimension_name=dimension_name,hierarchy_name=dimension_name)
attribute_names.reverse()

In [103]:

reordered_col = attribute_names + [hierarchy.name] + level_names[1:]
reordered_col

['By Vice President', 'H2 Reorg', 'Region', 'level002', 'level001', 'level000']

In [105]:
dataframe.loc[:,reordered_col]

,By Vice President,H2 Reorg,Region,level002,level001,level000
0,VP1,Scandinavia,Denmark,Scandinavia,Europe,World
1,VP2,Scandinavia,Norway,Scandinavia,Europe,World
2,VP1,Scandinavia,Sweden,Scandinavia,Europe,World
3,VP1,Benelux,Luxemburg,Benelux,Europe,World
4,VP2,Benelux,Netherlands,Benelux,Europe,World
5,VP2,Benelux,Belgium,Benelux,Europe,World
6,VP1,Islands,Great Britain,Islands,Europe,World
7,VP2,Islands,Ireland,Islands,Europe,World
8,VP1,Central Europe,France,Central Europe,Europe,World
9,VP2,Central Europe,Germany,Central Europe,Europe,World


In [110]:
c = tm1.chores.get('CreateProducts_ProductTypeSubsets')


In [133]:
c.dst_sensitivity

False

In [124]:
s = set([1,3])
s.update([1,2,3])

In [126]:
tm1.processes.search_string_in_code('ViewZeroOut')

['AA_Care_Sales_Taille',
 'BTA',
 'Care_Sales_Taille',
 'DL_Import_Sales_Test',
 'DL_Import_Sales_Test_ETA',
 'DL_Sales_Sales_Test',
 'Import_Sales_Tes_Adel',
 'LSA',
 'LSA_Mois',
 'Mensualisation_LSA',
 'Mensualisation_YGH',
 'SalesPlanDeleteData',
 'Transfer_Versions_Month',
 'YGH',
 '}izi.cube.data.copy',
 '}tp_add_user_security_from_file',
 '}tp_application_deploy',
 '}tp_custom_TI_update_TI_cube',
 '}tp_deploy_app_cubes_cleanup',
 '}tp_reset_state']

In [130]:
print(tm1.processes.get('DL_Import_Sales_Test_ETA').prolog_procedure)
print(tm1.processes.get('DL_Import_Sales_Test_ETA').metadata_procedure)
print(tm1.processes.get('DL_Import_Sales_Test_ETA').data_procedure)
print(tm1.processes.get('DL_Import_Sales_Test_ETA').epilog_procedure)



#****Begin: Generated Statements***
#****End: Generated Statements****

#######################################
# -- Import de données .csv vers cube Sales_Test
# -- ETA - Date: 28/03/2024
#######################################

# -- Déclaration des variables

vViewNamePurge= GetProcessName() | 'Purge';
vSubsetNamePurge = GetProcessName() | 'Purge';
vCubeCible ='Sales_Test';
zFile = 'Import_Sales_Test.csv';

# -- Définition du nom et de l'emplacement du fichier source
File = 'D:\Applications\Tm1\CarSales_Test\Fichier\' | zFile;


# -- Définition de la source de données
DataSourceNameForServer = File;


# -- Création de la vue de purge 
ViewDestroy(vCubeCible, vViewNamePurge);
ViewCreate (vCubeCible, vViewNamePurge,0);

vDim = 'Time';
SubsetDestroy(vDim,vSubsetNamePurge);
SubsetCreate(vDim,vSubsetNamePurge,0);
i=1;
iMax= DimSiz(vDim);
while (i<=iMax);
vElem= DimNm(vDim,i);
	IF (Ellev(vDim,vElem)=0 & Elisanc(vDim, pAnnee, vElem)=1);
		SubsetElementInsert (vDim,vSubsetNamePurge, vElem,1

In [55]:
tm1.cubes.delete('}PickList_}DimensionProperties')

<Response [204]>

In [4]:
if not(tm1.dimensions.exists('}Picklist')):
    picklist_dim = TM1py.Dimension('}Picklist')
    tm1.dimensions.create(picklist_dim)


True

In [ ]:
newcube = TM1py.Cube('}}PickList_DimensionProperties',[''])

In [18]:
tm1.cubes.get('}PickList_TEST')

Cube:{"Name": "}PickList_TEST", "Dimensions@odata.bind": ["Dimensions('Region')", "Dimensions('Indicateur')", "Dimensions('}PickList')"], "Rules": "SKIPCHECK;\n\n['FPM'] = S: IF( DB('TEST', !Region, 'CPM' ) @= '' & DB('TEST', !Region, 'AMP' ) @= ''  , 'Static::x' , 'Static::'); \n\n['CPM'] = S: IF( DB('TEST', !Region, 'FPM' ) @= '' & DB('TEST', !Region, 'AMP' ) @= '' , 'Static::x' , 'Static::'); \n\n['AMP'] = S: IF( DB('TEST', !Region, 'CPM' ) @= '' & DB('TEST', !Region, 'FPM' ) @= '' , 'Static::x' , 'Static::'); \n\nFEEDERS;"}

In [32]:
dimension_name = 'dimension_test'

In [48]:
selected_sync_option = tm1.cells.get_value(cube_name='}DimensionProperties',elements=f"{dimension_name},TOUPDATE",dimensions=['}Dimensions','}DimensionProperties'])
selected_sync_option
'elements' in selected_sync_option.lower()

True

In [45]:
from TM1py.Utils import Utils

Utils.get_all_servers_from_adminhost()

In [50]:
tm1.cells.write_value(value='Updated',cube_name='}DimensionProperties',element_tuple=(dimension_name,'TOUPDATE'),dimensions=['}Dimensions','}DimensionProperties'])

<Response [204]>

In [56]:
tm1_master = TM1Service(**config['tm1srv01'])
tm1_other = TM1Service(**config['tm1srv02'])

In [58]:
dimension_master = tm1_master.dimensions.get(dimension_name=dimension_name)                 # main dimension
tm1_other.dimensions.update(dimension_master)